Reference:  
https://huggingface.co/docs/transformers/v4.17.0/en/tasks/sequence_classification  
https://huggingface.co/course/chapter3/3?fw=pt

## Preparing Variables

In [2]:
# Optional
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
seq_length = 128
pad_approach = "max_length"
bert_huggingface = "indolem/indobertweet-base-uncased" #uncased because we're using lower casing for all text data
num_epoch = 3

files = {
    "train" : "/content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/4. Train, Test/train/train_4.csv",
    "eval" : "/content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/4. Train, Test/validation/validation_4.csv",
    "test" : "/content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/4. Train, Test/test/test_3.csv",
}

save_model_path = "/content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Model/IndoBERTweet/v1"

load_model = True
load_path = "/content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Model/IndoBERTweet/v1"

## Install Prerequisite Library

In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 30.4 MB/s 
     |████████████████████████████████| 6.6 MB 33.3 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
     |████████████████████████████████| 596 kB 73.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 19.5 MB/s 
     |████████████████████████████████| 1.1 MB 65.3 MB/s 
     |████████████████████████████████| 140 kB 3.1 MB/s 
     |████████████████████████████████| 212 kB 46.9 MB/s 
     |████████████████████████████████| 86 kB 6.9 MB/s 
     |████████████████████████████████| 127 kB 22.6 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████

## Preparing Datasets for Training
The Datasets must consist of "text" column name and "labels" column name
  
And for the "label" we need to process 0 as negative sentiment and 1 as positive sentiment

In [ ]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files=files)

Using custom data configuration default-86ecf9b7890fd58a


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-86ecf9b7890fd58a/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

## Tokenizing Datasets for Training and Evaluation

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(load_path if load_model else bert_huggingface, use_fast=True)

In [ ]:
def tokenize_text(dataset_row):
    return tokenizer(str(dataset_row["text"]), max_length=seq_length,
                     truncation=True, padding="max_length")
    
    # # Extract mapping between new and old indices
    # sample_map = tokenize.pop("overflow_to_sample_mapping")
    # for key, values in dataset_row.items():
    #     tokenize[key] = [values[i] for i in sample_map]
    # return tokenize
    
token_datasets = dataset.map(tokenize_text,remove_columns=["text"])

  0%|          | 0/9758 [00:00<?, ?ex/s]

  0%|          | 0/2440 [00:00<?, ?ex/s]

  0%|          | 0/536 [00:00<?, ?ex/s]

## Importing Models

In [ ]:
from transformers import BertForSequenceClassification 
model = BertForSequenceClassification . \
        from_pretrained(load_path if load_model else bert_huggingface, num_labels=2)

# The warning is expected because we're importing from BertForPreTraining Model

## Preparing Training

In [ ]:
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer

def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch",
                                  num_train_epochs=num_epoch)

trainer = Trainer(
    model,
    training_args,
    train_dataset=token_datasets["train"],
    eval_dataset=token_datasets["eval"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()
# Validation data => Seimbangin

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: raw. If raw are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9758
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3660


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.342000,0.338348,0.895492,0.897467
2,0.198600,0.411947,0.907787,0.905976
3,0.074500,0.529754,0.904508,0.901812


Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1000
Configuration saved in test-trainer/checkpoint-1000/config.json
Model weights saved in test-trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1000/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: raw. If raw are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num exam

Saving model checkpoint to test-trainer/checkpoint-1500
Configuration saved in test-trainer/checkpoint-1500/config.json
Model weights saved in test-trainer/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-2000
Configuration saved in test-trainer/checkpoint-2000/config.json
Model weights saved in test-trainer/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-2000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-2000/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: raw. If raw are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num

TrainOutput(global_step=3660, training_loss=0.2045771731704962, metrics={'train_runtime': 846.4099, 'train_samples_per_second': 34.586, 'train_steps_per_second': 4.324, 'total_flos': 1925578258652160.0, 'train_loss': 0.2045771731704962, 'epoch': 3.0})

In [ ]:
trainer.save_model(save_model_path)

Saving model checkpoint to /content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Model/IndoBERTweet/v1
Configuration saved in /content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Model/IndoBERTweet/v1/config.json
Model weights saved in /content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Model/IndoBERTweet/v1/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Model/IndoBERTweet/v1/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Model/IndoBERTweet/v1/special_tokens_map.json


Preidct Test Data

In [ ]:
# Data Test Lama

In [ ]:
prediction = trainer.predict(token_datasets["test"])
prediction

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: raw. If raw are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 536
  Batch size = 8


PredictionOutput(predictions=array([[ 3.5154011, -3.516872 ],
       [-3.4222887,  3.5927863],
       [-2.9704704,  3.4504619],
       ...,
       [-3.2462013,  3.5374622],
       [ 4.194733 , -4.2243447],
       [ 4.0952725, -3.9753063]], dtype=float32), label_ids=array([0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
    

In [ ]:
# Data Test Baru
dataset_lama =  load_dataset('csv', data_files={"test_lama" : "/content/drive/Shareddrives/Riset Sentimen Vaksin COVID: Yaudahlah/Riset/Data/4. Train, Test/test/test_lama.csv"})
tokenize_lama = dataset_lama.map(tokenize_text,remove_columns=["text"])

Using custom data configuration default-5333b95016b10e48


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-5333b95016b10e48/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?ex/s]

In [ ]:
prediction = trainer.predict(tokenize_lama["test_lama"])
prediction

***** Running Prediction *****
  Num examples = 80
  Batch size = 8


PredictionOutput(predictions=array([[-3.3292449 ,  3.6239014 ],
       [-3.6520574 ,  3.995167  ],
       [-3.8877127 ,  4.062715  ],
       [-3.6011932 ,  3.8855665 ],
       [-3.998148  ,  4.219166  ],
       [ 3.5877113 , -3.6588726 ],
       [-2.6878514 ,  2.7293003 ],
       [ 3.0923698 , -2.880236  ],
       [ 4.005708  , -4.0499573 ],
       [-2.6891396 ,  2.7884376 ],
       [-3.831788  ,  4.002137  ],
       [-3.475096  ,  3.7786174 ],
       [-3.8430696 ,  4.0504947 ],
       [-3.700349  ,  3.6983216 ],
       [-3.8403325 ,  3.862538  ],
       [-1.8954489 ,  2.137929  ],
       [-3.3986495 ,  3.6965954 ],
       [-2.6544683 ,  2.757974  ],
       [-3.662415  ,  3.7910159 ],
       [-2.9409795 ,  3.0546286 ],
       [ 2.6729639 , -2.6111944 ],
       [ 3.4923227 , -3.3716047 ],
       [-3.8322785 ,  4.018502  ],
       [-3.8356888 ,  4.0648904 ],
       [-3.576855  ,  3.8195422 ],
       [-3.3118334 ,  3.5215333 ],
       [-3.412827  ,  3.4610229 ],
       [-3.9155004 ,  4.16